# TASK 1 : Install Dependencies & Run a SparkSession


In [ ]:
#install pyspark
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=664861de43275ad61e2856876415d637f8b4b3d9e368c73897d5f2335ee86cb0
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
#create a sparksession
from pyspark.sql import SparkSession
spark =SparkSession.builder.appName("spark").getOrCreate()

# TASK 2 : Clone & Explore dataset

In [ ]:
#clone the dataset
! git clone https://github.com/education454/admission_dataset

Cloning into 'admission_dataset'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), 5.60 KiB | 5.60 MiB/s, done.


In [ ]:
#check the presence of dataset
!ls '/content/admission_dataset'


Admission_Predict_Ver1.1.csv


In [ ]:
#create a spark dataframe
df = spark.read.csv ('/content/admission_dataset',header=True,inferSchema=True)

In [ ]:
#display dataframe
df.show()

In [ ]:
#get the no.of rows & columns
print((df.count(),len(df.columns)))

(500, 9)


In [ ]:
#print schema
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [ ]:
#get the summary statistics
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|        Serial No|         GRE Score|      TOEFL Score|University Rating|               SOP|               LOR|              CGPA|          Research|    Chance of Admit|
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

# TASK 3 : Data Cleaning

In [ ]:
#drop the unnecessary column
df = df.drop('Serial no')

In [ ]:
#display the dataframe
df.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|      302|        102|                1|2.0|1.5| 8.0|       0|            0.5|
|      323|        108|                3

In [ ]:
#check for null values
for i in df.columns:
   print(i+":",df[df[i].isNull()].count())

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0


# TASK 4 : Correlation Analysis & Feature Selection

In [ ]:
# correlation analysis
for col in df.columns:
  print('Correlation to chance of admit col for {} is {}'.format(col,df.stat.corr('Chance of Admit',col)))

Correlation to chance of admit col for GRE Score is 0.8103506354632598
Correlation to chance of admit col for TOEFL Score is 0.7922276143050823
Correlation to chance of admit col for University Rating is 0.6901323687886892
Correlation to chance of admit col for SOP is 0.6841365241316723
Correlation to chance of admit col for LOR is 0.6453645135280112
Correlation to chance of admit col for CGPA is 0.882412574904574
Correlation to chance of admit col for Research is 0.5458710294711379
Correlation to chance of admit col for Chance of Admit is 1.0


In [ ]:
# feature selection
from pyspark.ml.feature import VectorAssembler
assembler =VectorAssembler(inputCols=['GRE Score','TOEFL Score','CGPA',],outputCol='features')

In [ ]:
#display dataframe
output_data = assembler.transform(df)

# TASK 5 : Build the Linear Regression Model

In [ ]:
#import Linearregression and create final data
from pyspark.ml.regression import LinearRegression
final_data = output_data.select('features','chance of admit')

In [ ]:
#print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- chance of admit: double (nullable = true)



In [ ]:
#split the dataset into training and testing set
train , test = final_data.randomSplit([0.7,0.3])

In [ ]:
#build & train the model
models = LinearRegression(featuresCol='features',labelCol='chance of admit')
model=models.fit(train)

In [ ]:
#get coefficients & intercept
print("coefficents:",model.coefficients)
print('intercept:',model.intercept)

coefficents: [0.002459634738797738,0.003095417657225585,0.14655688783927281]
intercept: -1.645795015868514


In [ ]:
#get summary of the model
summary =model.summary

In [ ]:
#print the rmse & r2 score
print('RMSE',summary.rootMeanSquaredError)
print('r2 score',summary.r2)

RMSE 0.06591676049037955
r2 score 0.795012759917565


# TASK 6 : Evaluate & Save the Model

In [ ]:
#transform on the test data
predictions = model.transform(test)

In [ ]:
#display the predictions
predictions.show(20)

+------------------+---------------+-------------------+
|          features|chance of admit|         prediction|
+------------------+---------------+-------------------+
| [294.0,93.0,7.36]|           0.46|0.44387013395704833|
| [295.0,96.0,7.34]|           0.47|0.45268488391073736|
| [297.0,98.0,7.67]|           0.59| 0.5121587616897443|
| [297.0,100.0,7.9]|           0.52| 0.5520576812072282|
|  [298.0,98.0,7.5]|           0.44| 0.4897037254958654|
|  [298.0,99.0,7.6]|           0.46| 0.5074548319370182|
|[298.0,100.0,7.95]|           0.58| 0.5618451603379893|
|[298.0,101.0,7.86]|           0.54| 0.5517504580896804|
| [299.0,94.0,7.34]|           0.42| 0.4563325875514772|
|[299.0,100.0,7.88]|           0.68| 0.5540458129280379|
|[299.0,100.0,7.89]|           0.59| 0.5555113818064308|
|[299.0,100.0,8.02]|           0.63| 0.5745637772255361|
| [299.0,106.0,8.4]|           0.64| 0.6488279005478133|
|  [300.0,97.0,8.1]|           0.65|  0.579461710019799|
| [300.0,98.0,8.02]|           

In [ ]:
#evaluate the model
from Pyspark.ml.evaluation import RegressionEvaulator
evaluator = RegressionEvaulator(predictioncol='prediction',labelcol='Chance of Admit',metricName='r2')

In [ ]:
#save the model

In [ ]:
#load the model